In [11]:
from pinecone import Pinecone, ServerlessSpec
import time
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Pinecone configuration
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = 'product-catalog-index'
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# Connect to the index
myindex = pc.Index(index_name)
time.sleep(1)

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Load environment variables from a .env file
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embed_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [13]:
from langchain_pinecone import PineconeVectorStore
# Initialize Pinecone (assuming `myindex` is already created and configured)
vectorstore = PineconeVectorStore(
    index=myindex,  # Pinecone index instance
    embedding=embed_model,  # Embeddings object
    text_key='Description'  # Field in the database to be embedded
)

In [14]:
query = "What is the price of DKNY Unisex Black & Grey Printed Medium Trolley Bag?"

vectorstore.similarity_search(
    query,  # our search query
    k=1  # return most relevant doc
)

[Document(id='10017413', metadata={'Gender': 'Unisex', 'Price': '500 USD', 'PrimaryColor': ' Black', 'ProductBrand': 'DKNY', 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag'}, page_content='Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer')]

In [15]:
import os
import google.generativeai as genai

# Load environment variables from a .env file
load_dotenv()

# Initialize the chat history
chat_history = []

# Define system_message globally or at the top of the notebook
system_message = (
    "If a query lacks a direct answer e.g. durability, generate a response based on related features. "
    "You are a helpful and respectful shop assistant who answers queries relevant only to the shop. "
    "Please answer all questions politely. Use a conversational tone, like you're chatting with someone, "
    "not like you're writing an email. If the user asks about anything outside of the shop data like if they ask "
    "something irrelevant, simply say, 'I can only provide answers related to the shop, sir."
)

# Function to generate a response from Google Gemini
def generate_answer(system_message, chat_history, prompt):
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model = genai.GenerativeModel('gemini-pro')

    # Append the new prompt to the chat history
    chat_history.append(f"User: {prompt}")

    # Combine the system message with the chat history
    full_prompt = f"{system_message}\n\n" + "\n".join(chat_history) + "\nAssistant:"

    # Generate the response and add it to the chat history
    response = model.generate_content(full_prompt).text
    chat_history.append(f"Assistant: {response}")
    
    return response

# Function to create the relevant passage from vectorstore
def get_relevant_passage(query, vectorstore):
    results = vectorstore.similarity_search(query, k=1)
    if results:
        metadata = results[0].metadata
        context = (
            f"Product Name: {metadata.get('ProductName', 'Not Available')}\n"
            f"Brand: {metadata.get('ProductBrand', 'Not Available')}\n"
            f"Price: {metadata.get('Price', 'Not Available')}\n"
            f"Color: {metadata.get('PrimaryColor', 'Not Available')}\n"
            f"Description: {results[0].page_content}"
        )
        return context
    return "No relevant results found"

# Function to create the prompt for the chatbot
def make_rag_prompt(query, context):
    return f"Query: {query}\n\nContext:\n{context}\n\nAnswer:"

In [16]:
def main():
    query = "What is the warranty of DKNY Unisex Black & Grey Printed Medium Trolley Bag? And please call me john when I ask you questions"

    # Retrieve relevant passage and create a prompt
    relevant_text = get_relevant_passage(query, vectorstore)
    prompt = make_rag_prompt(query, relevant_text)

    # Generate and print the final answer, maintaining chat history
    answer = generate_answer(system_message, chat_history, prompt)
    print("Answer:", answer)

    # Continue interaction (example of another question)
    query2 = "Can you tell me about the features of this bag?"
    relevant_text2 = get_relevant_passage(query2, vectorstore)
    prompt2 = make_rag_prompt(query2, relevant_text2)
    answer2 = generate_answer(system_message, chat_history, prompt2)
    print("Answer:", answer2)

if __name__ == "__main__":
    main()

Answer: Sure, John! The DKNY Unisex Black & Grey Printed Medium Trolley Bag comes with a 5-year warranty provided by the brand owner/manufacturer. Is there anything else I can help you with?
Answer: Sure, John! The Lino Perros Beige Solid Handheld Bag is a stylish and functional bag with a variety of features:
- It has a spacious main compartment with a zip closure to keep your belongings secure.
- There is also an external pocket and three inner pockets for easy organization of your smaller items.
- The bag has two handles for comfortable carrying.
- It does not come with a tablet sleeve

Is there anything else I can help you with, John?


In [17]:
# Third query
query3 = "How durable is the DKNY Unisex Black & Grey Printed Medium Trolley Bag?"
relevant_text3 = get_relevant_passage(query3, vectorstore)
prompt3 = make_rag_prompt(query3, relevant_text3)

# Generate and print the answer
answer3 = generate_answer(system_message, chat_history, prompt3)
print("Answer:", answer3)

Answer: I'm sorry, John, but I can't provide a direct answer to your question about the durability of the DKNY Unisex Black & Grey Printed Medium Trolley Bag based on the information available. However, I can tell you that the bag is made of high-quality materials and comes with a 5-year warranty from the manufacturer. This suggests that the bag is built to last and should withstand regular use.
